In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.output_parsers import JsonOutputKeyToolsParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a translator bot. Translate sentences from {source_language} to {target_language}.",
        ),
        (
            "human",
            "Translate this: {sentence}",
        ),
    ]
)

chat = ChatAnthropic(temperature=0.1, model_name="")

chain = prompt | chat | JsonOutputKeyToolsParser()

chain.invoke(
    {
        "source_language": "English",
        "target_language": "Italian",
        "sentence": "I love you, Langchain!",
    }
)

ValidationError: 1 validation error for JsonOutputKeyToolsParser
key_name
  field required (type=value_error.missing)